In [14]:
# torch包和d2l包默认安装在了d2l包下面
import torch
from IPython import display
from d2l import torch as d2l
import torch
import torchvision
import os
os.environ ["KMP_DUPLICATE_LIB_OK"] ="TRUE"
import matplotlib.pyplot as plt
from torch.utils import data
# torchvision 包含一些计算机视觉的相关库 transfrom包含图像处理的一些算法
from torchvision import transforms
from d2l import torch as d2l
batch_size = 256
def get_dataloader_workers():  #@save
    """使用4个进程来读取数据"""
    return 4

# 这里数据提前下好了 只需要读取本地文件即可
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="T://ProgramsData//data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="T://ProgramsData//data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=get_dataloader_workers()))


train_iter, test_iter = load_data_fashion_mnist(batch_size)

## 初始化模型参数

In [15]:
num_inputs = 784 
# 十个类别
num_outputs = 10
# 权重正态分布初始化 784 * 10
W = torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
# 偏置
b = torch.zeros(num_outputs,requires_grad=True)

In [16]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1,keepdim=True)
    return X_exp / partition


In [17]:
X = torch.normal(0,1,(2,5))
X_prob = softmax(X)
X_prob,X_prob.sum(1)

(tensor([[0.3935, 0.1623, 0.1716, 0.0743, 0.1983],
         [0.0528, 0.0068, 0.1162, 0.0676, 0.7566]]),
 tensor([1.0000, 1.0000]))

## 定义模型

In [18]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1,W.shape[0])),W) + b)

## 定义损失函数

In [19]:
y = torch.tensor([0,2])
y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
# [0,1]为行索引，y为列索引，y_hat是一个新张量
y_hat[[0,1],y]

tensor([0.1000, 0.5000])

In [20]:
# y_hat为对应的预测概率值 yi当中有一个为1 其余为0
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)),y])

cross_entropy(y_hat,y)

tensor([2.3026, 0.6931])

In [21]:
t = torch.normal(0,1,(2,5))
print(len(t))

2


In [22]:
# 返回数据
def accuracy(y_hat,y):
    # 二维张量且第二维分类>1 取概率最大的那个
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis = 1)
    # cmp是一个bool数组 y_hat预测值和实际值y相同是true
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [23]:
accuracy(y_hat, y) / len(y)

0.5

In [ ]:
# Accumulator类对多个变量进行累加
class Accumulator:
    def __init__(self,n):
        self.data = [0.0] * n
    
    # 将传入的参数和累加器当中的数据逐个对应相加
    def add(self,*args):
        self.data = [a + float(b) for a,b in zip(self.data,args)]
    
    def reset(self):
        self.data = [0.0] * len(self.data)
    
    def _getitem__(self,idx):
        return self.data[idx]


def evaluate_accuracy(net, data_iter):
    # 评价任意模型net精度
    # 判断net是不是moudle的实例 pt当中的神经网络基类
    if isinstance(net,torch.nn.Module):
        # 进入评估模式 梯度不会被计算
        net.eval()
    # 正确预测数 和 预测总数累加
    metric = Accumulator(2)
    # 接下来的代码不需要计算梯度 节省内存加快计算
    with torch.no_grad():
        # X 输入特征 y真实标签
        for X,y in data_iter:
            # 将准确率accuracy(net(X),y) 与 样本数量y.numel加到累加器中
            # y.numel返回y当中的数量
            metric.add(accuracy(net(X),y),y.numel())
    # 整个数据集上的准确率 正确预测数 / 预测种数
    return metric[0] / metric[1]


